# Review Instructions

Please review the MSv4 `field_and_source_xds` schema and the XRADIO interface (`ps['MSv4_name'].VISIBILITY.field_and_source_xds`). Note that the PS (processing set) interface or the main_xds should not be reviewed.

The `field_and_source_xds` schema specification: https://docs.google.com/spreadsheets/d/14a6qMap9M5r_vjpLnaBKxsR9TF4azN5LVdOxLacOX-s/edit#gid=1658760192

## Preparatory Material
Go over Xarray nomenclature and selection syntax:
- https://docs.xarray.dev/en/latest/user-guide/terminology.html
- https://docs.xarray.dev/en/latest/user-guide/indexing.html

MSv2 and CASA documentation:
- MSv2 schema: https://casacore.github.io/casacore-notes/229.pdf
- MSv3 schema: https://casacore.github.io/casacore-notes/264.pdf
- Ephemeris Data in CASA: https://casadocs.readthedocs.io/en/latest/notebooks/external-data.html#Ephemeris-Data

## `field_and_source_xds` Schema
The FIELD, SOURCE, and EPHEMERIS tables in the MSv2 contain closely related information:
- **FIELD**: Field position for a source.
- **SOURCE**: Information about the source being observed (position, proper motion, etc.)
- **EPHEMERIS**: Ephemerides of the source.

These can be combined into a single dataset for MSv4 because it consists of a single field and consequently a single source[^1].

### Use Cases
The use cases considered during the design of the schema were:
- Single field observation (type=standard).
- Mosaic observation (type=standard).
- Ephemeris observation (type=ephemeris).
- Mosaic Ephemeris observation (type=ephemeris).

To satisfy these use cases, two types of `field_and_source_xds` were created: standard and ephemeris. The main difference is that the ephemeris type has a `FIELD_PHASE_OFFSET` data variable that is relative to the `SOURCE_POSITION/SOURCE_DIRECTION` data variable (contains the ephemerides and has a time axis), while the standard type has `FIELD_PHASE/DELAY/REFERENCE_CENTERS` and `SOURCE_POSITION` (has no time axis). The `SOURCE_POSITION/DIRECTION` is kept separate from the `FIELD_PHASE_OFFSET/CENTER` so that the intent `OBSERVE_TARGET#OFF_SOURCE` is supported and the ephemeris can be easily changed.

## Key Questions to Answer
### Schema Questions
- 1.1) Are there missing use cases?
- 1.2) Is all the information present needed for offline processing?
  - 1.2.1) Are there data variables we need to add (for example, the JPL Horizons data has additional information such as the North pole position angle, etc., see [CASA Ephemeris Data](https://casadocs.readthedocs.io/en/latest/notebooks/external-data.html#Ephemeris-Data))?
  - 1.2.2) In some CASA ephemeris tables, there are table keywords such as `VS_CREATE`, `VS_DATE`, `VS_TYPE`, `VS_VERSION`, `MJD0`, `dMJD`, `earliest`, `latest`, `radii`, `meanradm`, `orb_per`, `rot_per`. Do we need any of these?
- 1.3) Is there a use case where the `FIELD_PHASE_CENTER` and `FIELD_DELAY_CENTER` would differ (i.e., do we need to store both)?
- 1.4) For interferometer observations, do we need to store the `FIELD_REFERENCE_CENTER` or can this be omitted (will still be present for Single dish)?
- 1.5) The ephemeris data is recorded in degrees, AU, and MJD. Should these be converted to radians, meters, and time (Unix)? Note that each data variable has measurement information attached to it. For example:
```Python
  frame: ICRS
  type: sky_coord
  units: ['deg', 'deg', 'AU']
```

- 1.6) For ephemeris observations, should we add the SOURCE_PROPER_MOTION if available?
- 1.7) Is the name `field_and_source_xds` sufficiently descriptive?
- 1.8) Should we also add the DOPPLER table information to the schema (if so, any idea where we can get an MSv2 with a DOPPLER table)?
- 1.9) Any naming suggestions or data layout?
- 1.10) Are the data variable descriptions in the schema spreadsheet correct?
- 1.11) What measures (https://docs.google.com/spreadsheets/d/14a6qMap9M5r_vjpLnaBKxsR9TF4azN5LVdOxLacOX-s/edit#gid=1504318014) should we attach to each of the following data variables

  - NORTH_POLE_POSITION_ANGLE (quantity?)
  - NORTH_POLE_ANGULAR_DISTANCE (quantity?)
  - SUB_OBSERVER_DIRECTION (earth_location?)
  - SUB_SOLAR_POSITION (earth_location?)
  - HELIOCENTRIC_RADIAL_VELOCITY (quantity?)
  - OBSERVER_PHASE_ANGLE (quantity?)
- 1.12) Can NORTH_POLE_POSITION_ANGLE and NORTH_POLE_ANGULAR_DISTANCE be combined into a single data variable?

### XRADIO
2.1) After reviewing the XARRAY documentation and the descriptions of the data variables in the `field_and_source_xds` schema, do you find the XARRAY interface intuitive and easy to use?



[^1]: This is inhereted from MSv2 that only allows a single source per field [https://casacore.github.io/casacore-notes/229.pdf, p35], though a source can appear in more than one field.


# Environment instructions

It is recommended to use the conda environment manager to create a clean, self-contained runtime where xradio and all its dependencies can be installed:

```bash
conda create --name xradio python=3.11 --no-default-packages
conda activate xradio
```

Clone the repository, checkout the review branch and do a local install:

```bash
git clone https://github.com/casangi/xradio.git
git checkout 162-create-combined-field-source-and-ephemeris-dataset
cd xradio
pip install -e .
```

# ALMA Example

An ephemeris mosaic observation of the sun.

ALMA archive file downloaded: https://almascience.nrao.edu/dataPortal/2022.A.00001.S_uid___A002_X1003af4_X75a3.asdm.sdm.tar 

- Project: 2022.A.00001.S
- Member ous id (MOUS): uid://A001/X3571/X130
- Group ous id (GOUS): uid://A001/X3571/X131

CASA commands used to create the dataset:
```python
importasdm(asdm='uid___A002_X1003af4_X75a3.asdm.sdm',vis='uid___A002_X1003af4_X75a3.ms',asis='Ephemeris Antenna Station Receiver Source CalAtmosphere CalWVR',bdfflags=True,with_pointing_correction=True,convert_ephem2geo=True)

importasdm(asdm='22A-347.sb41163051.eb41573599.59642.62832791667.asdm',vis='22A-347.sb41163051.eb41573599.59642.62832791667.ms',asis='Ephemeris Antenna Station Receiver Source',with_pointing_correction=True,convert_ephem2geo=True)

mstransform(vis='ALMA_uid___A002_X1003af4_X75a3.split.ms',outputvis='ALMA_uid___A002_X1003af4_X75a3.split.avg.ms',createmms=False,timeaverage=True,timebin='2s',timespan='scan',reindex=True)

import numpy as np
tb.open('ALMA_uid___A002_X1003af4_X75a3.split.avg.ms::FLAG_CMD',nomodify=False)
tb.removerows(np.arange(tb.nrows())) # tb.removerows(np.arange(76401))
tb.flush()
tb.done()

tb.open('ALMA_uid___A002_X1003af4_X75a3.split.avg.ms::POINTING',nomodify=False)
tb.removerows(np.arange(tb.nrows()))   #tb.removerows(np.arange(5617115))
tb.flush()
tb.done()
```


## Download Data

In [1]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set
from xradio.vis.read_processing_set import read_processing_set
import graphviper

graphviper.utils.data.download(file="ALMA_uid___A002_X1003af4_X75a3.split.avg.ms")

[2024-07-03 16:50:59,271]    DEBUG      xradio:  Initializing downloader with 1 threads.


 Download List                                
 ───────────────────────────────────────────── 
  ALMA_uid___A002_X1003af4_X75a3.split.avg.ms

[2024-07-03 16:50:59,294]     INFO      xradio:  File exists: ALMA_uid___A002_X1003af4_X75a3.split.avg.ms 


# Start Dask cluster 
Choose an approriate number of cores and memory_limit (this is per core).

In [2]:
from graphviper.dask.client import local_client

viper_client = local_client(cores=4, memory_limit="4GB")
viper_client

[2024-07-03 16:50:59,383]     INFO      xradio:  Checking parameter values for client.local_client 
[2024-07-03 16:50:59,383]     INFO      xradio:  Module path: /Users/jsteeb/Dropbox/graphviper/ 
[2024-07-03 16:50:59,383]     INFO      xradio:  Searching /Users/jsteeb/Dropbox/graphviper/src/graphviper/config/ for configuration file, please wait ... 
[2024-07-03 16:50:59,384]    DEBUG      xradio:  Result: /Users/jsteeb/Dropbox/graphviper/src/graphviper/config
[2024-07-03 16:50:59,384]    DEBUG      xradio:  PARAMETER_CONFIG_PATH: /Users/jsteeb/Dropbox/graphviper/src/graphviper/config
[2024-07-03 16:50:59,384]    DEBUG      xradio:  /Users/jsteeb/Dropbox/graphviper/src/graphviper/config/client.param.json
[2024-07-03 16:50:59,388]  WARNING      client:  It is recommended that the local cache directory be set using the dask_local_dir parameter. 
[2024-07-03 16:51:00,065]     INFO      client:  Created client <MenrvaClient: 'tcp://127.0.0.1:62267' processes=4 threads=4, memory=14.90 GiB> 

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 14.90 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:62267,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 14.90 GiB
Comm: tcp://127.0.0.1:62279,Total threads: 1
Dashboard: http://127.0.0.1:62283/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:62270,


# Convert dataset

In [3]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

in_file = "ALMA_uid___A002_X1003af4_X75a3.split.avg.ms"
out_file = "ALMA_uid___A002_X1003af4_X75a3.split.avg.zarr"

convert_msv2_to_processing_set(
    in_file=in_file,
    out_file=out_file,
    partition_scheme="ddi_intent_field",
    parallel=True,
    overwrite=True,
    ephemeris_interpolate=True
)

[None]
[2024-07-03 16:51:01,142]     INFO    worker_1:  field_and_source_xds: interpolating the time coordinate from 2 to 12 points 
[2024-07-03 16:51:01,160]     INFO    worker_2:  field_and_source_xds: interpolating the time coordinate from 2 to 12 points 
[2024-07-03 16:51:01,203]     INFO    worker_3:  field_and_source_xds: interpolating the time coordinate from 2 to 9 points 
[2024-07-03 16:51:01,366]     INFO    worker_1:  field_and_source_xds: interpolating the time coordinate from 2 to 12 points 
[2024-07-03 16:51:01,484]     INFO    worker_3:  field_and_source_xds: interpolating the time coordinate from 2 to 2 points 
[2024-07-03 16:51:01,573]     INFO    worker_0:  field_and_source_xds: interpolating the time coordinate from 2 to 9 points 
[2024-07-03 16:51:01,784]     INFO    worker_0:  field_and_source_xds: interpolating the time coordinate from 2 to 6 points 
[2024-07-03 16:51:02,090]     INFO    worker_3:  field_and_source_xds: interpolating the time coordinate from 2 to 

In [4]:
import numpy as np

phase_deg = np.mod(np.linspace(0 ,720, 19), 360) - 180
print(phase_deg)
print( np.unwrap(phase_deg, period=360))

[-180. -140. -100.  -60.  -20.   20.   60.  100.  140. -180. -140. -100.
  -60.  -20.   20.   60.  100.  140. -180.]
[-180. -140. -100.  -60.  -20.   20.   60.  100.  140.  180.  220.  260.
  300.  340.  380.  420.  460.  500.  540.]


# Inspect Processing Set

In [5]:
import pandas as pd

# Set the maximum number of rows displayed before scrolling
pd.set_option("display.max_rows", 1000)

from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set("ALMA_uid___A002_X1003af4_X75a3.split.avg.zarr")
ps.summary()

,name,ddi,intent,field_id,field_name,start_frequency,end_frequency,shape,field_coords
0,ALMA_uid___A002_X1003af4_X75a3.split.avg_310,0,OBSERVE_TARGET#ON_SOURCE,10,Sun_10,2.480000e+11,2.480000e+11,"(12, 51, 1, 2)",Ephemeris
1,ALMA_uid___A002_X1003af4_X75a3.split.avg_328,0,OBSERVE_TARGET#ON_SOURCE,28,Sun_10,2.480000e+11,2.480000e+11,"(8, 51, 1, 2)",Ephemeris
2,ALMA_uid___A002_X1003af4_X75a3.split.avg_317,0,OBSERVE_TARGET#ON_SOURCE,17,Sun_10,2.480000e+11,2.480000e+11,"(12, 51, 1, 2)",Ephemeris
3,ALMA_uid___A002_X1003af4_X75a3.split.avg_321,0,OBSERVE_TARGET#ON_SOURCE,21,Sun_10,2.480000e+11,2.480000e+11,"(12, 51, 1, 2)",Ephemeris
4,ALMA_uid___A002_X1003af4_X75a3.split.avg_90,0,"CALIBRATE_ATMOSPHERE#HOT,CALIBRATE_WVR#HOT",0,Sun_10,2.480000e+11,2.480000e+11,"(3, 51, 1, 2)",Ephemeris
5,ALMA_uid___A002_X1003af4_X75a3.split.avg_319,0,OBSERVE_TARGET#ON_SOURCE,19,Sun_10,2.480000e+11,2.480000e+11,"(12, 51, 1, 2)",Ephemeris
6,ALMA_uid___A002_X1003af4_X75a3.split.avg_326,0,OBSERVE_TARGET#ON_SOURCE,26,Sun_10,2.480000e+11,2.480000e+11,"(12, 51, 1, 2)",Ephemeris
7,ALMA_uid___A002_X1003af4_X75a3.split.avg_988,2,OBSERVE_TARGET#ON_SOURCE,28,Sun_10,2.299609e+11,2.300547e+11,"(6, 1326, 7, 2)",Ephemeris
8,ALMA_uid___A002_X1003af4_X75a3.split.avg_986,2,OBSERVE_TARGET#ON_SOURCE,26,Sun_10,2.299609e+11,2.300547e+11,"(9, 1326, 7, 2)",Ephemeris
9,ALMA_uid___A002_X1003af4_X75a3.split.avg_972,2,OBSERVE_TARGET#ON_SOURCE,12,Sun_10,2.299609e+11,2.300547e+11,"(9, 1326, 7, 2)",Ephemeris


In [6]:
ps['ALMA_uid___A002_X1003af4_X75a3.split.avg_310'].VISIBILITY.attrs["field_and_source_xds"].is_ephemeris

True

# Inspect field_and_source_xds: Standard Use case (non-ephemeris)

In [7]:
standard_field_and_source_xds = ps[
    "ALMA_uid___A002_X1003af4_X75a3.split.avg_902"
].VISIBILITY.field_and_source_xds.load()  # Load the data into memory
standard_field_and_source_xds

<xarray.Dataset> Size: 160B
Dimensions:                 (sky_dir_label: 2, line_name: 1)
Coordinates:
  * line_name               (line_name) <U22 88B 'Single_Continuum(ID=0)'
  * sky_dir_label           (sky_dir_label) <U3 24B 'ra' 'dec'
Data variables:
    FIELD_PHASE_CENTER      (sky_dir_label) float64 16B -2.579 -0.1374
    LINE_REST_FREQUENCY     (line_name) float64 8B 2.3e+11
    LINE_SYSTEMIC_VELOCITY  (line_name) float64 8B 0.0
    SOURCE_DIRECTION        (sky_dir_label) float64 16B -2.579 -0.1374
Attributes:
    code:          none
    field_code:    none
    field_id:      2
    field_name:    J1408-0752
    is_ephemeris:  False
    source_id:     2
    source_name:   J1408-0752

In [8]:
standard_field_and_source_xds.attrs[
    "field_id"
]  # How to access field_and_source_xds attributes.

2

In [9]:
standard_field_and_source_xds.FIELD_PHASE_CENTER  # How to access field_and_source_xds datavariables. standard_field_and_source_xds['FIELD_PHASE_CENTER'] can also be used.

<xarray.DataArray 'FIELD_PHASE_CENTER' (sky_dir_label: 2)> Size: 16B
array([-2.57897986, -0.13742852])
Coordinates:
  * sky_dir_label  (sky_dir_label) <U3 24B 'ra' 'dec'
Attributes:
    frame:    ICRS
    type:     sky_coord
    units:    ['rad', 'rad']

In [10]:
standard_field_and_source_xds.FIELD_PHASE_CENTER.attrs  # How to access field_and_source_xds datavariables measures information stored in the attributes.

{'frame': 'ICRS', 'type': 'sky_coord', 'units': ['rad', 'rad']}

# Inspect field_and_source_xds: Ephemeris Use case (Mosaic) with line

In [11]:
ephemeris_field_and_source_xds = ps[
    "ALMA_uid___A002_X1003af4_X75a3.split.avg_963"
].VISIBILITY.field_and_source_xds.load()  # Load the data into memory
ephemeris_field_and_source_xds

<xarray.Dataset> Size: 1kB
Dimensions:                       (time: 9, sky_pos_label: 3, line_name: 1,
                                   ellipsoid_pos_label: 3, sky_dir_label: 2)
Coordinates:
  * ellipsoid_pos_label           (ellipsoid_pos_label) <U4 48B 'lon' ... 'dist'
  * line_name                     (line_name) <U22 88B 'Single_Continuum(ID=0)'
  * sky_dir_label                 (sky_dir_label) <U3 24B 'ra' 'dec'
  * sky_pos_label                 (sky_pos_label) <U4 48B 'ra' 'dec' 'dist'
  * time                          (time) float64 72B 1.667e+09 ... 1.667e+09
Data variables:
    FIELD_PHASE_CENTER            (time, sky_pos_label) float64 216B -2.624 ....
    HELIOCENTRIC_RADIAL_VELOCITY  (time) float64 72B 0.0 0.0 0.0 ... 0.0 0.0 0.0
    LINE_REST_FREQUENCY           (line_name) float64 8B 2.3e+11
    LINE_SYSTEMIC_VELOCITY        (line_name) float64 8B 0.0
    NORTH_POLE_ANGULAR_DISTANCE   (time) float64 72B 542.7 542.7 ... 547.5 547.5
    NORTH_POLE_POSITION_ANGLE     (time) float64 72B 14.5 14.5 ... 14.63 14.63
    OBSERVATION_POSITION          (ellipsoid_pos_label) float64 24B 0.0 0.0 0.0
    OBSERVER_PHASE_ANGLE          (time) float64 72B 0.0 0.0 0.0 ... 0.0 0.0 0.0
    SOURCE_POSITION               (time, sky_pos_label) float64 216B 3.659 .....
    SOURCE_RADIAL_VELOCITY        (time) float64 72B -205.3 -205.3 ... -203.0
    SUB_OBSERVER_POSITION         (time, ellipsoid_pos_label) float64 216B 3....
Attributes:
    code:          none
    field_code:    none
    field_id:      3
    field_name:    Sun_10
    is_ephemeris:  True
    source_id:     0
    source_name:   Sun_10

In [12]:
ephemeris_field_and_source_xds.FIELD_PHASE_CENTER

<xarray.DataArray 'FIELD_PHASE_CENTER' (time: 9, sky_pos_label: 3)> Size: 216B
array([[-2.62435062e+00, -2.11302878e-01,  1.48765976e+11],
       [-2.62435023e+00, -2.11303019e-01,  1.48765975e+11],
       [-2.62434984e+00, -2.11303159e-01,  1.48765974e+11],
       [-2.62430930e+00, -2.11317776e-01,  1.48765875e+11],
       [-2.62430891e+00, -2.11317916e-01,  1.48765874e+11],
       [-2.62430852e+00, -2.11318056e-01,  1.48765873e+11],
       [-2.62426798e+00, -2.11332673e-01,  1.48765774e+11],
       [-2.62426759e+00, -2.11332813e-01,  1.48765773e+11],
       [-2.62426720e+00, -2.11332954e-01,  1.48765772e+11]])
Coordinates:
  * sky_pos_label  (sky_pos_label) <U4 48B 'ra' 'dec' 'dist'
  * time           (time) float64 72B 1.667e+09 1.667e+09 ... 1.667e+09
Attributes:
    frame:    ICRS
    type:     sky_coord
    units:    ['rad', 'rad', 'm']

In [13]:
ephemeris_field_and_source_xds.SOURCE_POSITION

<xarray.DataArray 'SOURCE_POSITION' (time: 9, sky_pos_label: 3)> Size: 216B
array([[ 3.65892193e+00, -2.11176957e-01,  1.48765976e+11],
       [ 3.65892232e+00, -2.11177098e-01,  1.48765975e+11],
       [ 3.65892271e+00, -2.11177238e-01,  1.48765974e+11],
       [ 3.65896325e+00, -2.11191855e-01,  1.48765875e+11],
       [ 3.65896364e+00, -2.11191995e-01,  1.48765874e+11],
       [ 3.65896403e+00, -2.11192136e-01,  1.48765873e+11],
       [ 3.65900457e+00, -2.11206752e-01,  1.48765774e+11],
       [ 3.65900496e+00, -2.11206892e-01,  1.48765773e+11],
       [ 3.65900535e+00, -2.11207033e-01,  1.48765772e+11]])
Coordinates:
  * sky_pos_label  (sky_pos_label) <U4 48B 'ra' 'dec' 'dist'
  * time           (time) float64 72B 1.667e+09 1.667e+09 ... 1.667e+09
Attributes:
    frame:    ICRS
    type:     sky_coord
    units:    ['rad', 'rad', 'm']

In [14]:
ephemeris_field_and_source_xds.SOURCE_POSITION.sel(sky_pos_label="dec")

<xarray.DataArray 'SOURCE_POSITION' (time: 9)> Size: 72B
array([-0.21117696, -0.2111771 , -0.21117724, -0.21119185, -0.211192  ,
       -0.21119214, -0.21120675, -0.21120689, -0.21120703])
Coordinates:
    sky_pos_label  <U4 16B 'dec'
  * time           (time) float64 72B 1.667e+09 1.667e+09 ... 1.667e+09
Attributes:
    frame:    ICRS
    type:     sky_coord
    units:    ['rad', 'rad', 'm']

In [15]:
ps[
    "ALMA_uid___A002_X1003af4_X75a3.split.avg_963"
]

<xarray.Dataset> Size: 3MB
Dimensions:                     (time: 9, baseline_id: 1326, frequency: 7,
                                 polarization: 2, uvw_label: 3)
Coordinates:
    baseline_antenna1_id        (baseline_id) int32 5kB dask.array<chunksize=(1326,), meta=np.ndarray>
    baseline_antenna2_id        (baseline_id) int32 5kB dask.array<chunksize=(1326,), meta=np.ndarray>
  * baseline_id                 (baseline_id) int64 11kB 0 1 2 ... 1324 1325
  * frequency                   (frequency) float64 56B 2.3e+11 ... 2.301e+11
  * polarization                (polarization) <U2 16B 'XX' 'YY'
  * time                        (time) float64 72B 1.667e+09 ... 1.667e+09
  * uvw_label                   (uvw_label) <U1 12B 'u' 'v' 'w'
Data variables:
    EFFECTIVE_INTEGRATION_TIME  (time, baseline_id) float64 95kB dask.array<chunksize=(9, 1326), meta=np.ndarray>
    FLAG                        (time, baseline_id, frequency, polarization) bool 167kB dask.array<chunksize=(9, 1326, 7, 2), meta=np.ndarray>
    TIME_CENTROID               (time, baseline_id) float64 95kB dask.array<chunksize=(9, 1326), meta=np.ndarray>
    UVW                         (time, baseline_id, uvw_label) float64 286kB dask.array<chunksize=(9, 1326, 3), meta=np.ndarray>
    VISIBILITY                  (time, baseline_id, frequency, polarization) complex64 1MB dask.array<chunksize=(9, 1326, 7, 2), meta=np.ndarray>
    WEIGHT                      (time, baseline_id, frequency, polarization) float32 668kB dask.array<chunksize=(9, 1326, 7, 2), meta=np.ndarray>
Attributes:
    data_groups:     {'base': {'flag': 'FLAG', 'uvw': 'UVW', 'visibility': 'V...
    ddi:             2
    intent:          OBSERVE_TARGET#ON_SOURCE
    partition_info:  {'field_id': 3, 'field_name': 'Sun_10', 'intent': 'OBSER...
    pointing_xds:    <xarray.Dataset> Size: 0B\nDimensions:  ()\nData variabl...
    weather_xds:     <xarray.Dataset> Size: 138kB\nDimensions:         (stati...
    antenna_xds:     <xarray.Dataset> Size: 9kB\nDimensions:        (antenna_...

In [16]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

in_file = "VLASS3.2.sb45755730.eb46170641.60480.16266136574.split.v6.ms"
out_file = "VLASS3.2.sb45755730.eb46170641.60480.16266136574.split.v6.zarr"

convert_msv2_to_processing_set(
    in_file=in_file,
    out_file=out_file,
    partition_scheme="ddi_intent_scan",
    parallel=False,
    overwrite=True,
)